In [1]:
# Clear memory
%reset -f

In [44]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import ipywidgets
ipywidgets.Widget.close_all()

import SimpleITK
import datetime
import gc # Garbage collected
import numpy as np
import sys, os
import time
import tifffile
import matplotlib.pyplot as plt
import scipy
from tkinter import Tk
from tkinter.filedialog import askopenfilename, askopenfilenames, askdirectory
import torch
import skimage

this_path = os.getcwd()
print(this_path)


import h5py
# import File
from scripts import *
import wrapper_ASTRA


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/global/u2/e/eboigne/tomoTools_als832/tomoTools


In [3]:
path_save = '/global/homes/e/eboigne/cfs_als/2022_wood/'
case = 'run24_birch_lowHeat_test2'

In [12]:
ind_slice = 300

list_cases = sorted([e for e in os.listdir(path_save+case) if not 'probe' in e])
print(list_cases)

slices = []
for ind_case, this_case in enumerate(list_cases[:-1]):
    slices.append(File(path_save+case+'/'+this_case+'/b_movingRegisteredToStatic/').read(ind_slice))
    print(ind_case)

slices = np.array(slices)
print(slices.shape)
File(path_save+case+'/probe_slice_'+str(ind_slice).zfill(4)).saveTiffStack(slices)

['01', '02', '03', '04', '05', '06', '07']
0
1
2
3
4
5
(6, 1280, 1280)


In [4]:
scan_to_test = '07'

# Load data
tic = time.time()
data = File(path_save+case+'/'+scan_to_test+'/b_movingRegisteredToStatic/').readAll()
print('Loading data took: '+str(time.time()-tic))
print(data.shape)

Loading data took: 5.758517026901245
(698, 1280, 1280)


In [ ]:
threshold = 0.10 # Linear attenuation threshold [/cm]

bin_factor_dilatation = 1 # Binning before smoothing, as an option to speed things up for large kernels
filter_half_width = 3 #  After binning (equal to sigma for gaussian filter)

# kernel = custom_3d_kernel_sphere(filter_half_width)
kernel = custom_3d_gaussian_filter(filter_half_width)
# filter_half_width = (kernel.shape[0]-1)/2

tic = time.time()
if bin_factor_dilatation > 1:
    data_smoothed = fast_pytorch_bin_3d(data,bin_factor_dilatation, chunk_size = 71)
else:
    data_smoothed = data

print(data_smoothed.shape)
data_smoothed = apply_3d_image_processing_on_subvolumes(data_smoothed, fast_pytorch_convolution, kernel_array = kernel, chunk_size_max = (500, 500, 500), overlap = 3*filter_half_width)
print(data_smoothed.shape)

if bin_factor_dilatation > 1:
    data_smoothed = skimage.transform.rescale(data_smoothed, bin_factor_dilatation, multichannel=False)
print('Smoothing data took: '+str(time.time()-tic))

mask_solid = data_smoothed > threshold
File(path_save+case+'/'+scan_to_test+'/b_movingRegisteredToStatic_maskSolid').saveTiffStack(mask_solid, type = 'bool')

# Save smoothed data
# File(path_save+case+'/'+scan_to_test+'/b_movingRegisteredToStatic_smoothed_bin'+str(bin_factor_dilatation)+'x_FHW_'+str(filter_half_width)).saveTiffStack(data_smoothed)

(698, 1280, 1280)
	 Processing 18 chunks of size [349 426 426] .........

In [60]:
print(time.time())

1658277426.121251
